In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.corpus import *
from nltk.tokenize import *
import time

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aakul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
Yahoo_News = pd.read_csv('Yahoo_News_Scraped_0501.csv', index_col = 0)
Yahoo_News

,URL_LINK,DATES,ARTICLE_TITLE,ARTICLE_CONTENT
0,https://techcrunch.com/2020/05/01/valve-drops-...,2020-05-01T22:06:02,Valve drops VR support for macOS,Valve is calling it quits on macOS support for...
1,https://finance.yahoo.com/news/stock-market-ne...,2020-05-01T20:08:37.000Z,Stock market news live updates: Dow starts May...,Stocks were lower Friday after the latest batc...
2,https://finance.yahoo.com/news/what-is-smartph...,2020-05-01T14:08:12.000Z,What is smartphone contact tracing and how doe...,"Welcome to Tech Support, a segment where I, Da..."
3,https://finance.yahoo.com/news/warren-buffett-...,2020-05-01T11:54:13.000Z,Why Warren Buffett thinks about baseball legen...,One of the world’s most legendary and respecte...
4,https://finance.yahoo.com/news/the-month-stock...,2020-05-01T10:05:09.000Z,The month stocks broke from reality: Morning B...,"Friday, May 1, 2020Get the Morning Brief sent ..."
5,https://www.marketwatch.com/story/dow-futures-...,"\n Published: May 1, 2020 at 9:27 p.m. ET\n",Dow closes 600 points lower to wipe out weekly...,Stocks closed sharply lower Friday in an ugly ...
6,https://www.barrons.com/articles/why-big-tech-...,"\n May 1, 2020 7:36 pm ET\n",Why Big Tech Is Beating the Market. 5 Things W...,"This copy is for your personal, non-commercial..."
7,https://realmoney.thestreet.com/jim-cramer/jim...,2020-05-01T22:06:02,"Jim Cramer: Let's Talk About Amazon, Advanced ...","Let's us talk about the Triple A's. No, not th..."
8,https://finance.yahoo.com/news/video-warren-bu...,2020-05-01T22:59:52.000Z,Video: Warren Buffett's Market Indicator Rises...,"On Friday, the day before Berkshire Hathaway I..."
9,https://www.barrons.com/articles/dow-jones-ind...,"\n May 1, 2020 5:21 pm ET\n",The Dow Tumbled 622 Points Because Investors N...,"This copy is for your personal, non-commercial..."


In [3]:
Yahoo_News.dtypes

URL_LINK           object
DATES              object
ARTICLE_TITLE      object
ARTICLE_CONTENT    object
dtype: object

In [5]:
stop_words = nltk.corpus.stopwords.words('english')

In [6]:
Yahoo_News.ARTICLE_CONTENT = Yahoo_News.ARTICLE_CONTENT.astype(str)
new_article_content = []

for content in Yahoo_News['ARTICLE_CONTENT']:
    content = content.lower()
    content = content.lstrip(' ')
    
    sentence = ''
    tokenized_words = word_tokenize(content)
    
    for token in tokenized_words:
        if token not in stop_words:
            token = re.sub('[^.,a-zA-Z0-9 \n\.]', ' ', token)
            
            sentence += token
            sentence += ' '
            
    new_article_content.append(sentence)

new_article_content

['valve calling quits macos support virtual reality platform . valve employee posted update company   steamvr forums , noting   steamvr ended macos support team focus windows linux.   apple introduced   metal vr   back june 2017 highlighted partnership valve . time , valve pushing vr platform working htc vive system   fast forward 2020 valve high end headset released highly anticipated title   half life   alyx.   impacts developers actual gamers.almost games supported mac , even apple   highest end macbook pros failed meet minimum specification requirements oculus steamvr . folks upload point , valve   recent hardware survey showcases 4   gamers platform use macos begin , suggesting pretty small sliver actual gamers even fit mac owning vr user venn diagram.game developers building vr content mac likely enjoyed ability develop test single machine . apple aggressively chases professionals high priced gear like mac pro imac pro ,   great look major software platform decides   worth effort

In [7]:
post = []
neut = []
neg = []
comp = []

for transformed_content in new_article_content:
    score = SIA().polarity_scores(transformed_content)
    post.append(score['pos'])
    neut.append(score['neu'])
    neg.append(score['neg'])
    comp.append(score['compound'])

In [8]:
Yahoo_News['TRANSFORMED_ARTICLE_CONTENT'] = new_article_content
Yahoo_News['POSITIVE_VALUE'] = post
Yahoo_News['NEUTRAL_VALUE'] = neut
Yahoo_News['NEGATIVE_VALUE'] = neg
Yahoo_News['COMPOUND_VALUE'] = comp

Yahoo_News

,URL_LINK,DATES,ARTICLE_TITLE,ARTICLE_CONTENT,TRANSFORMED_ARTICLE_CONTENT,POSITIVE_VALUE,NEUTRAL_VALUE,NEGATIVE_VALUE,COMPOUND_VALUE
0,https://techcrunch.com/2020/05/01/valve-drops-...,2020-05-01T22:06:02,Valve drops VR support for macOS,Valve is calling it quits on macOS support for...,valve calling quits macos support virtual real...,0.164,0.790,0.046,0.9959
1,https://finance.yahoo.com/news/stock-market-ne...,2020-05-01T20:08:37.000Z,Stock market news live updates: Dow starts May...,Stocks were lower Friday after the latest batc...,stocks lower friday latest batch corporate ear...,0.077,0.779,0.144,-0.9961
2,https://finance.yahoo.com/news/what-is-smartph...,2020-05-01T14:08:12.000Z,What is smartphone contact tracing and how doe...,"Welcome to Tech Support, a segment where I, Da...","welcome tech support , segment , dan howley , ...",0.100,0.821,0.079,0.3699
3,https://finance.yahoo.com/news/warren-buffett-...,2020-05-01T11:54:13.000Z,Why Warren Buffett thinks about baseball legen...,One of the world’s most legendary and respecte...,"one world legendary respected investors , wa...",0.225,0.710,0.065,0.9935
4,https://finance.yahoo.com/news/the-month-stock...,2020-05-01T10:05:09.000Z,The month stocks broke from reality: Morning B...,"Friday, May 1, 2020Get the Morning Brief sent ...","friday , may 1 , 2020get morning brief sent di...",0.089,0.807,0.104,-0.8625
5,https://www.marketwatch.com/story/dow-futures-...,"\n Published: May 1, 2020 at 9:27 p.m. ET\n",Dow closes 600 points lower to wipe out weekly...,Stocks closed sharply lower Friday in an ugly ...,stocks closed sharply lower friday ugly start ...,0.099,0.797,0.104,0.6249
6,https://www.barrons.com/articles/why-big-tech-...,"\n May 1, 2020 7:36 pm ET\n",Why Big Tech Is Beating the Market. 5 Things W...,"This copy is for your personal, non-commercial...","copy personal , non commercial use . order pre...",0.141,0.763,0.096,0.9601
7,https://realmoney.thestreet.com/jim-cramer/jim...,2020-05-01T22:06:02,"Jim Cramer: Let's Talk About Amazon, Advanced ...","Let's us talk about the Triple A's. No, not th...","let s us talk triple s . , batteries , motor...",0.203,0.717,0.081,0.9971
8,https://finance.yahoo.com/news/video-warren-bu...,2020-05-01T22:59:52.000Z,Video: Warren Buffett's Market Indicator Rises...,"On Friday, the day before Berkshire Hathaway I...","friday , day berkshire hathaway inc. s nyse...",0.098,0.852,0.050,0.9643
9,https://www.barrons.com/articles/dow-jones-ind...,"\n May 1, 2020 5:21 pm ET\n",The Dow Tumbled 622 Points Because Investors N...,"This copy is for your personal, non-commercial...","copy personal , non commercial use . order pre...",0.109,0.829,0.062,0.9705


In [9]:
Yahoo_News.DATES = Yahoo_News.DATES.astype(str)

for date in range(len(Yahoo_News['DATES'])):
    match = re.search(r'\w{3}\s\d{1,2}\,\s\d{4}', Yahoo_News['DATES'][date])
    try:
        mdate = dt.datetime.strptime(match[0], '%b %d, %Y')
        Yahoo_News['DATES'][date] = mdate
    except:
        pass

for date in range(len(Yahoo_News['DATES'])):
    match = re.search(r'\d{2}\/\d{2}\/\d{4}', Yahoo_News['DATES'][date])
    try:
        mdate = dt.datetime.strptime(match[0], '%m/%d/%Y')
        Yahoo_News['DATES'][date] = mdate
    except:
        pass
    
for date in range(len(Yahoo_News['DATES'])):
    match = re.search(r'\d{4}\-\d{2}\-\d{2}', Yahoo_News['DATES'][date])
    try:
        mdate = dt.datetime.strptime(match[0], '%Y-%m-%d')
        Yahoo_News['DATES'][date] = mdate
    except:
        pass

<ipython-input-9-4b35d4306584>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Yahoo_News['DATES'][date] = mdate


TypeError: expected string or bytes-like object

In [ ]:
Yahoo_News

In [ ]:
Yahoo_News.DATES = Yahoo_News.DATES.astype('datetime64')

In [ ]:
Yahoo_News

In [ ]:
Yahoo_News.info()

In [ ]:
Yahoo_News.to_csv('Yahoo_News_Sentiment')

In [11]:
Yahoo_News['ARTICLE_CONTENT'][0]

'Valve is calling it quits on macOS support for its virtual reality platform. A Valve employee posted an update to the company’s SteamVR forums, noting that “SteamVR has ended macOS support so our team can focus on Windows and Linux.”Apple  introduced “Metal for VR” back in June 2017 and highlighted a partnership with Valve. At the time, Valve was pushing VR as a platform and working with HTC on its Vive system; fast-forward to 2020 and Valve has its own high-end headset and has just released its highly-anticipated title “Half Life: Alyx.”This impacts developers more so than actual gamers.Almost no games supported Mac, and even Apple’s highest-end MacBook Pros failed to meet minimum specification requirements for Oculus or SteamVR. As the folks at Upload point out, Valve’s recent hardware survey showcases that just 4% of gamers on the platform use macOS to begin with, suggesting a pretty small sliver of actual gamers even fit in the Mac-owning VR user Venn diagram.Game developers build

In [10]:
Yahoo_News['TRANSFORMED_ARTICLE_CONTENT'][0]

'valve calling quits macos support virtual reality platform . valve employee posted update company   steamvr forums , noting   steamvr ended macos support team focus windows linux.   apple introduced   metal vr   back june 2017 highlighted partnership valve . time , valve pushing vr platform working htc vive system   fast forward 2020 valve high end headset released highly anticipated title   half life   alyx.   impacts developers actual gamers.almost games supported mac , even apple   highest end macbook pros failed meet minimum specification requirements oculus steamvr . folks upload point , valve   recent hardware survey showcases 4   gamers platform use macos begin , suggesting pretty small sliver actual gamers even fit mac owning vr user venn diagram.game developers building vr content mac likely enjoyed ability develop test single machine . apple aggressively chases professionals high priced gear like mac pro imac pro ,   great look major software platform decides   worth effort.